In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# 1. 데이터 불러오기

In [2]:
# 데이터 불러오기
import pandas as pd
import numpy as np
import random as rnd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# 머신러닝
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
#data 불러오기
train=pd.read_csv("/kaggle/input/titanic/train.csv") #학습용 데이터
test=pd.read_csv("/kaggle/input/titanic/test.csv") #테스트용(제출용) 데이터
train_test=[train,test]

# 2. data 분석

In [5]:
# 변수 보기
print(train.columns.values)

#변수설명
''' 
PassengerId : 각 승객의 고유 번호
Survived : 생존 여부(종속 변수)
    0 = 사망
    1 = 생존
Pclass : 객실 등급 - 승객의 사회적, 경제적 지위
    1st = Upper
    2nd = Middle
    3rd = Lower
Name : 이름
Sex : 성별
Age : 나이
SibSp : 동반한 Sibling와 Spouse의 수
Parch : 동반한 Parent와 Child의 수
Ticket : 티켓의 고유넘버
Fare : 티켓의 요금
Cabin : 객실 번호
Embarked : 승선한 항
    C = Cherbourg
    Q = Queenstown
    S = Southampton'''

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


' \nPassengerId : 각 승객의 고유 번호\nSurvived : 생존 여부(종속 변수)\n    0 = 사망\n    1 = 생존\nPclass : 객실 등급 - 승객의 사회적, 경제적 지위\n    1st = Upper\n    2nd = Middle\n    3rd = Lower\nName : 이름\nSex : 성별\nAge : 나이\nSibSp : 동반한 Sibling와 Spouse의 수\nParch : 동반한 Parent와 Child의 수\nTicket : 티켓의 고유넘버\nFare : 티켓의 요금\nCabin : 객실 번호\nEmbarked : 승선한 항\n    C = Cherbourg\n    Q = Queenstown\n    S = Southampton'

In [6]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
train.info() #데이터 type과 null유무확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# 데이터분석

age, cabin,embarked 값은 null값이 존재 >> 나중에 값을 넣어줘야함

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [9]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# 3. Data 전처리

In [10]:
train_test_data = [train, test]

# Sex 전처리
sex_mapping = {"male":0, "female":1}
for dataset in train_test_data:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)

# SibSp & Parch 전처리
for dataset in train_test_data:
    # 가족수 = 형제자매 + 부모님 + 자녀 + 본인
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['IsAlone'] = 1
    
    # 가족수 > 1이면 동승자 있음
    dataset.loc[dataset['FamilySize'] > 1, 'IsAlone'] = 0
    
# Embarked 전처리
for dataset in train_test_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

embarked_mapping = {'S':0, 'C':1, 'Q':2}
for dataset in train_test_data:
    dataset['Embarked'] = dataset['Embarked'].map(embarked_mapping)
    
# Name 전처리
for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract('([\w]+)\.', expand=False)

for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].apply(lambda x: 0 if x=="Mr" else 1 if x=="Miss" else 2 if x=="Mrs" else 3 if x=="Master" else 4)

In [11]:
# 결측치 제거
for dataset in train_test_data:
    dataset['Age'].fillna(dataset.groupby("Title")["Age"].transform("median"), inplace=True)
    
# min-max 스케일링
#각 데이터에서 최소값을 뺀 뒤 범위(max - min)로 나누어 모든 데이터를 0~1 사이에 오도록 만드는 방법이다. 상한과 하한이 확실히 정해지는 장점
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
for dataset in train_test_data:
    array = dataset['Age'].values.reshape(-1,1) # 2D array로 변환
    scaler.fit(array) # 스케일링에 필요한 값(최소값, range 등) 계산
    dataset['AgeScale'] = pd.Series(scaler.transform(array).reshape(-1)) # 스케일링 후 series로 추가
    
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone,Title,AgeScale
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,2,0,0,0.271174
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,2,0,2,0.472229
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1,1,1,0.321438
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,2,0,2,0.434531
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,1,1,0,0.434531


In [12]:
# 결측치 제거
for dataset in train_test_data:
    dataset["Fare"].fillna(dataset.groupby("Pclass")["Fare"].transform("median"), inplace=True)
    
# 표준화
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
for dataset in train_test_data:
    array = dataset['Fare'].values.reshape(-1,1)
    scaler.fit(array)
    dataset['FareScale'] = pd.Series(scaler.transform(array).reshape(-1))
    
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone,Title,AgeScale,FareScale
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,2,0,0,0.271174,-0.502445
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1,2,0,2,0.472229,0.786845
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1,1,1,0.321438,-0.488854
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,2,0,2,0.434531,0.420730
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,1,1,0,0.434531,-0.486337


In [13]:
# 전처리가 끝난 특성들 제거
drop_column = ['Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin']

for dataset in train_test_data:
    dataset = dataset.drop(drop_column, axis=1, inplace=True)

# 상관계수 계산
corr_matrix = train.corr()
corr_matrix["Survived"].sort_values(ascending=False)

Survived       1.000000
Sex            0.543351
Title          0.414088
FareScale      0.257307
Embarked       0.106811
FamilySize     0.016639
PassengerId   -0.005007
AgeScale      -0.078698
IsAlone       -0.203367
Pclass        -0.338481
Name: Survived, dtype: float64

In [14]:
drop_column = ['FamilySize']

for dataset in train_test_data:
    dataset = dataset.drop(drop_column, axis=1, inplace=True)
    
train.head()

,PassengerId,Survived,Pclass,Sex,Embarked,IsAlone,Title,AgeScale,FareScale
0,1,0,3,0,0,0,0,0.271174,-0.502445
1,2,1,1,1,1,0,2,0.472229,0.786845
2,3,1,3,1,0,1,1,0.321438,-0.488854
3,4,1,1,1,0,0,2,0.434531,0.420730
4,5,0,3,0,0,1,0,0.434531,-0.486337


# 4. Data 분할 및 모델 설정

In [15]:
# 훈련을 위한 train, target 분할
drop_column2 = ['PassengerId', 'Survived']
train_data = train.drop(drop_column2, axis=1)
target = train['Survived']

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score # 교차 검증 패키지
from sklearn.metrics import mean_squared_error

clf = DecisionTreeClassifier()
clf.fit(train_data, target) # 모델 훈련
prediction = clf.predict(train_data) # 예측
tree_mse = mean_squared_error(target, prediction) # 오차 계산
tree_rmse = np.sqrt(tree_mse) 
print("훈련 데이터 점수 :", tree_rmse)

# 교차 검증
scores = cross_val_score(clf, train_data, target,
                        scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

# 점수 표시 함수 작성
def display_scores(scores):
    print("교차검증 점수:", scores)
    print("평균:", scores.mean())
    print("표준편차:", scores.std())

display_scores(tree_rmse_scores)

훈련 데이터 점수 : 0.138129235668458
교차검증 점수: [0.52704628 0.44971901 0.56089816 0.48575205 0.41053541 0.47404546
 0.39661489 0.51929079 0.44971901 0.44971901]
평균: 0.47233400786264773
표준편차: 0.04930691414954007


In [16]:
score_list=[]
kfold_score_list=[]
for i in range(1,11):
    clf = DecisionTreeClassifier(max_depth = i)
    clf.fit(train_data, target)
    prediction = clf.predict(train_data)
    tree_mse = mean_squared_error(target, prediction)
    tree_rmse = np.sqrt(tree_mse)
    scores = cross_val_score(clf, train_data, target,
                        scoring="neg_mean_squared_error", cv=10)
    tree_rmse_scores = np.sqrt(-scores)
    score_list.append(tree_rmse)
    kfold_score_list.append(tree_rmse_scores.mean())

In [17]:
score_data = {'훈련 세트 점수' : score_list, '교차 검증 점수' : kfold_score_list}
score_data_frame = pd.DataFrame(score_data, index=range(1,11))
score_data_frame

,훈련 세트 점수,교차 검증 점수
1,0.466619,0.465002
2,0.459346,0.457680
3,0.408935,0.417801
4,0.383439,0.412000
5,0.376051,0.420415
6,0.356123,0.412391
7,0.321333,0.419620
8,0.299644,0.427550
9,0.265908,0.421851
10,0.227217,0.438541


In [18]:
score_list=[]
kfold_score_list=[]
for i in range(1,11):
    clf = DecisionTreeClassifier(max_depth = 4, min_samples_leaf = i)
    clf.fit(train_data, target)
    prediction = clf.predict(train_data)
    tree_mse = mean_squared_error(target, prediction)
    tree_rmse = np.sqrt(tree_mse)
    scores = cross_val_score(clf, train_data, target,
                        scoring="neg_mean_squared_error", cv=10)
    tree_rmse_scores = np.sqrt(-scores)
    score_list.append(tree_rmse)
    kfold_score_list.append(tree_rmse_scores.mean())

In [19]:
score_data = {'훈련 세트 점수' : score_list, '교차 검증 점수' : kfold_score_list}
score_data_frame = pd.DataFrame(score_data, index=range(1,11))
score_data_frame

,훈련 세트 점수,교차 검증 점수
1,0.383439,0.412000
2,0.383439,0.412000
3,0.386355,0.412000
4,0.386355,0.412000
5,0.387805,0.412057
6,0.389249,0.410752
7,0.390688,0.412694
8,0.400617,0.411195
9,0.400617,0.412288
10,0.400617,0.412288


# Model 학습

In [20]:
# 모델 훈련
clf = DecisionTreeClassifier(max_depth=4, min_samples_leaf=6)
clf.fit(train_data, target)
test_data = test.drop("PassengerId", axis=1)
predict = clf.predict(test_data)

# 예측 결과 저장
submission = pd.DataFrame({
    'PassengerId' : test['PassengerId'],
    'Survived' : predict})

submission.to_csv('submission_titanic.csv', index=False)